In [4]:
!pip install -q tfds-nightly

In [2]:
!pip install -q tensorflow-hub

In [5]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.0.0
Eager mode:  True
Hub version:  0.8.0
GPU is NOT AVAILABLE


In [6]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Shuffling and writing examples to C:\Users\Alan\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incomplete5JG2D7\imdb_reviews-train.tfrecord


Shuffling and writing examples to C:\Users\Alan\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incomplete5JG2D7\imdb_reviews-test.tfrecord


Shuffling and writing examples to C:\Users\Alan\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incomplete5JG2D7\imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to C:\Users\Alan\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [11]:
#選取10段電影評論
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_labels_batch

<tf.Tensor: id=208, shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)>

In [12]:
train_examples_batch

<tf.Tensor: id=207, shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion

In [13]:
#如何代表文字？
#在模型中使用多少層？
#每層使用多少個隱藏單元？
#將句子轉換為嵌入向量。使用預先訓練文本嵌入模型重tensorflow hub
#使用hub.KerasLayer遷入模型
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: id=388, shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [20]:
model=tf.keras.Sequential([
    hub_layer,
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(16,activation='relu'),
    tf.keras.layers.Dense(1)
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               2688      
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 16)                1040      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 412,021
Trainable params: 412,021
Non-trainable params: 0
_________________________________________________________________


In [21]:
#損失函數和優化器
#由於這是一個二元分類問題，並且該模型輸出logit
#（具有線性激活的單個單位層），因此我們將使用binary_crossentropy損失函數。
model.compile(optimizer='adam',
             loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy'])

In [22]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 3s 108ms/step - loss: 0.4371 - accuracy: 0.7701 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
30/30 [==============================] - 3s 92ms/step - loss: 0.2268 - accuracy: 0.9093 - val_loss: 0.3212 - val_accuracy: 0.8662
Epoch 3/20
30/30 [==============================] - 3s 90ms/step - loss: 0.1883 - accuracy: 0.9267 - val_loss: 0.3349 - val_accuracy: 0.8581
Epoch 4/20
30/30 [==============================] - 3s 91ms/step - loss: 0.1645 - accuracy: 0.9375 - val_loss: 0.3183 - val_accuracy: 0.8710
Epoch 5/20
30/30 [==============================] - 3s 92ms/step - loss: 0.1457 - accuracy: 0.9443 - val_loss: 0.3252 - val_accuracy: 0.8733
Epoch 6/20
30/30 [==============================] - 3s 91ms/step - loss: 0.1324 - accuracy: 0.9503 - val_loss: 0.3436 - val_accuracy: 0.8638
Epoch 7/20
30/30 [==============================] - 3s 92ms/step - loss: 0.1238 - accuracy: 0.9527 - val_loss: 0.3434 - val_accuracy: 0.8726
Epoc

In [23]:
result= model.evaluate(test_data.batch(512))

49/49 [==============================] - 2s 45ms/step - loss: 0.6986 - accuracy: 0.8464


In [27]:
print(next(iter(test_data.batch(1))))

(<tf.Tensor: id=9347, shape=(1,), dtype=string, numpy=
array([b"There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven't laughed this hard since I saw THE FULL MONTY. (And, even then, I don't think I laughed quite this hard... So to speak.) Tukel's talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there's none of the over-the-top scenery chewing one might've expected from a film like this). DING-A-LING-LESS is a film whose time has co

In [46]:
a=['Adam Trevor is the hero of the film. Although it is not a typical example of handsome, several scenes of singing and dancing, collapse and want to escape, and seeing Qian Yan opened the show, and they all showed up. In particular, he is very interesting to play with Jonathan Price. You say one word to me, from time to time inscriptions, correct lines, and the boundary between illusion and reality is blurred. In the end, there was a kind of warmth and inheritance, which was touching.']
b=['The theme and idea of the movie is good, but it is too boring to make, and it is not worthy of being a comedy. The director wanted to imitate "Crazy Stone" to make a multi-line narrative film, which integrates all kinds of errors and coincidences, and coincidence and accident. All in all, the film is different from the vomit-like disgusting nausea in "Aunt Li Cha", but it is a rough embarrassment that is simple enough to save up the scraps thrown by others in the TV series. Allen is losing his audience, step by step, and wasting Wang Zhi ’s stature and acting skills of Ren Dahua and Jin Shijie.']
a=tf.constant(b)
print(a)
predictions=model.predict(a)

tf.Tensor([b'The theme and idea of the movie is good, but it is too boring to make, and it is not worthy of being a comedy. The director wanted to imitate "Crazy Stone" to make a multi-line narrative film, which integrates all kinds of errors and coincidences, and coincidence and accident. All in all, the film is different from the vomit-like disgusting nausea in "Aunt Li Cha", but it is a rough embarrassment that is simple enough to save up the scraps thrown by others in the TV series. Allen is losing his audience, step by step, and wasting Wang Zhi \xe2\x80\x99s stature and acting skills of Ren Dahua and Jin Shijie.'], shape=(1,), dtype=string)


In [47]:
if predictions[0][0]<0:
    print('negative ')
else:
    print('postive')

negative 
